In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
from nltk.tokenize import RegexpTokenizer

In [2]:
df_all = pd.read_excel(r'D:\TEST Codingan\Sistem-Temu-Kembali-Informasi\Riset Jupiter Notebook\Data\df_final.xlsx').head(1000)

# EDA

In [3]:
df_all = df_all.drop(columns = ['Unnamed: 0'])

In [4]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   divisions          917 non-null    object
 1   title              1000 non-null   object
 2   abstract           758 non-null    object
 3   subjects           1000 non-null   object
 4   publication        498 non-null    object
 5   publisher          863 non-null    object
 6   creators           999 non-null    object
 7   date               1000 non-null   object
 8   keywords           181 non-null    object
 9   uri                1000 non-null   object
 10  new_abstract       758 non-null    object
 11  text_raw           1000 non-null   object
 12  language           1000 non-null   object
 13  language_accuracy  1000 non-null   object
 14  fulltext           1000 non-null   object
 15  fulltext_stopword  1000 non-null   object
 16  text_stemmed       1000 non-null   object
d

In [5]:
pd.DataFrame(df_all[['title', 'abstract']])

,title,abstract
0,TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,Metil metsulfuron merupakan senyawa aktif yang...
1,Media Pembelajaran Digital sebagai Sumber Bela...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...
2,Strategi Keberlanjutan Jasa Air Bersih Berbasi...,Persoalan penyediaan jasa air bersih merupakan...
3,Metode Sexing Spermatozoa XY dan Aplikasi Inse...,NaN
4,PENGARUH BAHAN PELLETING TERHADAP PERKECAMBAHA...,Bawang merah merupakan komoditas pertanian pen...
...,...,...
995,Analisis Kelayakan Finansial Usaha Burung Puyu...,Unggas penghasil telur terbesar kedua setelah ...
996,Buku Teknologi Pengelolaan Agen Biologis Hara,NaN
997,Teknologi Pengelolaan Agen Biologis Hara (TPABH),NaN
998,PENGARUH KECERDASAN EMOSIONAL DAN KEMAMPUAN AW...,Pembelajaran online menyisakan banyak permasal...


In [ ]:
topik_riset = {'phew'}

for i in range(len(df_all['fokus_riset'])):
  topik_riset.add(df_all['fokus_riset'].iloc[i])

In [ ]:
topik_riset

{'Integrasi Fokus Riset Energi-Energi Baru dan Terbarukan',
 'Kebencanaan',
 'Kemaritiman',
 'Kesehatan-Obat',
 'Material Maju',
 'Pangan-Pertanian',
 'Pertahanan dan Keamanan',
 'Sosial Humaniora, Seni Budaya, Pendidikan',
 'Teknologi Informasi dan Komunikasi',
 'Transportasi',
 nan,
 'phew'}

In [ ]:
temporal = []

for i in range(len(df_all['fokus_riset'])):
  if df_all['fokus_riset'].iloc[i] == 'Kesehatan-Obat':
    temporal.append(df_all.iloc[i])

In [ ]:
temporal = []

for i in range(len(df_all['fokus_riset'])):
  if pd.isnull(df_all['fokus_riset'].iloc[i]):
    temporal.append(df_all.iloc[i])

In [ ]:
len(temporal)

1240

In [ ]:
df_temp = pd.DataFrame(temporal)
df_temp[['title', 'fokus_riset']]

In [ ]:
df_all['fokus_riset'].iloc[3921] = 'Kesehatan-Obat'
df_all['fokus_riset'].iloc[3926] = 'Sosial Humaniora, Seni Budaya, Pendidikan'

# Mesin Pencari

In [3]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         1000 non-null   int64 
 1   divisions          917 non-null    object
 2   title              1000 non-null   object
 3   abstract           758 non-null    object
 4   subjects           1000 non-null   object
 5   publication        498 non-null    object
 6   publisher          863 non-null    object
 7   creators           999 non-null    object
 8   date               1000 non-null   object
 9   keywords           181 non-null    object
 10  uri                1000 non-null   object
 11  new_abstract       758 non-null    object
 12  text_raw           1000 non-null   object
 13  language           1000 non-null   object
 14  language_accuracy  1000 non-null   object
 15  fulltext           1000 non-null   object
 16  fulltext_stopword  1000 non-null   object
 

In [4]:
df_all['title'].iloc[5]

'PENGENALAN BUNGA KRISAN POTONG DI DESA SUNGAI \nLANGKA KAB. PESAWARAN SEBAGAI KOMODITAS BARU DI \nDESA AGROWISATA'

In [5]:
# Inisialisasi TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

In [6]:
# Contoh teks
text1 = 'Tingkat METIL METSULFURON dalam COpedoda dhipa sp hahahaha'
text2 = 'Tingkat METIL METSULFURON dalam TOKSISITAS MASKULINITAS COPEPODA Daphnia sp.'

# Inisialisasi TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Transformasi teks menjadi vektor TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform([text1, text2])

# Hitung similaritas kosinus antara kedua teks
cosine_similarities = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

# Tampilkan hasil similaritas
print("Similaritas antara teks 1 dan teks 2:", cosine_similarities[0][0])

Similaritas antara teks 1 dan teks 2: 0.42113589132819584


In [18]:
dicari = 'Media Pembelajaran Digital'
dicari = dicari.lower()

In [8]:
def remove_stopwords(text):
    factory = StopWordRemoverFactory()
    stopwords = factory.get_stop_words()

    result = []
    for x in  text.split():
        if x not in stopwords:
            result.append(x)

    hasil_stopword = " ".join(result)

    return hasil_stopword

In [9]:
def stem(text):
    factory2 = StemmerFactory()
    stemmer = factory2.create_stemmer()
    return stemmer.stem(text)

In [19]:
dicari = stem(dicari)
dicari

'media ajar digital'

In [20]:
from tqdm import tqdm

df_temu = []
similaritas = []

for i in tqdm(range(len(df_all['text_stemmed']))):
  temp = df_all['text_stemmed'].iloc[i]

  # Transformasi teks menjadi vektor TF-IDF
  tfidf_matrix = tfidf_vectorizer.fit_transform([dicari, temp])

  # Hitung similaritas kosinus antara kedua teks
  cosine_similarities = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

  if cosine_similarities[0][0] > 0.1:
    df_temu.append(df_all.iloc[i])
    similaritas.append(cosine_similarities)

df_temu = pd.DataFrame(df_temu)
df_temu['similaritas'] = similaritas

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:04<00:00, 222.87it/s]


In [21]:
len(df_temu)

51

In [22]:
df_temu[['title', 'similaritas','abstract']].sort_values(by=['similaritas'], ascending=False)

,title,similaritas,abstract
1,Media Pembelajaran Digital sebagai Sumber Bela...,[[0.5970104031840774]],Kegiatan pembelajaran siswa SMA dan SMK mutlak...
267,Pengaruh Kemandirian Belajar dan Literasi Digi...,[[0.33629419908221336]],Penelitian ini dilatarbelakangi oleh masalah y...
717,Pelatihan Canva dalam Pembuatan Media Pembelaj...,[[0.2886301664442466]],Canva merupakan suatu tools desain grafis untu...
622,Pemanfaatan e-learning platform bags guru sent...,[[0.25214998317955867]],Perkembangan Abad 21 mengubah seluruh tatanan ...
350,PENYULUHAN HUKUM DAMPAK PENYALAHGUNAAN MEDIA S...,[[0.23309397363400536]],Di era globalisasi sekarang ini banyak sekali ...
7,Pengaruh Literasi Digital Terhadap Etika Berme...,[[0.22897168705235882]],Tujuan dari penelitian ini adalah untuk menget...
633,Pengaruh Model Talking Stick dan Media Audio V...,[[0.22368546382120325]],Pendidik diharapkan mampu menerapkan model pem...
691,Pelatihan Perancangan Media Pembelajaran Mengg...,[[0.21050479099859587]],Pemerintah Indonesia melalui Kementerian Pendi...
887,PENDAMPINGAN KEMITRAAN UMKM GO DIGITAL BAGI NE...,[[0.20456184329793164]],UMKM go digital merupakan program pemerintah s...
937,Teori Belajar Behavioristik,[[0.19884443692969783]],Teori belajar behavioristik dikenal juga denga...


In [23]:
tokenizer = RegexpTokenizer(r'\w+')

topik = []

for index in range(len(df_temu)):
  # Ambil dataset contoh (dalam kasus ini, dataset 20 newsgroups)
  teks = df_temu['fulltext_stopword'].iloc[index]
  data = tokenizer.tokenize(teks)

  # Proses vektorisasi teks dengan TF-IDF
  tfidf_vectorizer = TfidfVectorizer(max_features=5000)
  tfidf_matrix = tfidf_vectorizer.fit_transform(data)

  # Terapkan LSA dengan TruncatedSVD
  num_topics = 4  # Jumlah topik yang ingin diidentifikasi
  lsa_model = TruncatedSVD(n_components=num_topics, random_state=42)
  lsa_topic_matrix = lsa_model.fit_transform(tfidf_matrix)

  # Ambil kata-kata kunci untuk setiap topik
  terms = tfidf_vectorizer.get_feature_names_out()
  top_keywords = 1  # Jumlah kata kunci untuk setiap topik (HANYA BISA DIAMBIL 1)

  kata_kunci = []

  for i, topic in enumerate(lsa_model.components_):
      top_keywords_idx = topic.argsort()[-top_keywords:][::-1]
      keywords = [terms[key_idx] for key_idx in top_keywords_idx]
      kata_kunci.append(keywords[0])

  topik.append(kata_kunci)

df_temu['topik'] = topik

In [24]:
df_temu[['title', 'similaritas','abstract','keywords','topik']].sort_values(by=['similaritas'], ascending=False)

,title,similaritas,abstract,keywords,topik
1,Media Pembelajaran Digital sebagai Sumber Bela...,[[0.5970104031840774]],Kegiatan pembelajaran siswa SMA dan SMK mutlak...,NaN,"[pembelajaran, sejarah, siswa, media]"
267,Pengaruh Kemandirian Belajar dan Literasi Digi...,[[0.33629419908221336]],Penelitian ini dilatarbelakangi oleh masalah y...,NaN,"[belajar, penelitian, kemandirian, literasi]"
717,Pelatihan Canva dalam Pembuatan Media Pembelaj...,[[0.2886301664442466]],Canva merupakan suatu tools desain grafis untu...,NaN,"[media, pembuatan, pembuatan, canva]"
622,Pemanfaatan e-learning platform bags guru sent...,[[0.25214998317955867]],Perkembangan Abad 21 mengubah seluruh tatanan ...,Keterampilan abad 21; guru abad 21; teknologi;...,"[guru, pembelajaran, abad, 21]"
350,PENYULUHAN HUKUM DAMPAK PENYALAHGUNAAN MEDIA S...,[[0.23309397363400536]],Di era globalisasi sekarang ini banyak sekali ...,NaN,"[sosial, media, informasi, penyuluhan]"
7,Pengaruh Literasi Digital Terhadap Etika Berme...,[[0.22897168705235882]],Tujuan dari penelitian ini adalah untuk menget...,NaN,"[penelitian, digital, literasi, etika]"
633,Pengaruh Model Talking Stick dan Media Audio V...,[[0.22368546382120325]],Pendidik diharapkan mampu menerapkan model pem...,NaN,"[hasil, model, belajar, penelitian]"
691,Pelatihan Perancangan Media Pembelajaran Mengg...,[[0.21050479099859587]],Pemerintah Indonesia melalui Kementerian Pendi...,NaN,"[pembelajaran, android, guru, pelatihan]"
887,PENDAMPINGAN KEMITRAAN UMKM GO DIGITAL BAGI NE...,[[0.20456184329793164]],UMKM go digital merupakan program pemerintah s...,"UMKM Go Digital, New Entrepreneur, Make-Up Art...","[umkm, digital, sosial, artist]"
937,Teori Belajar Behavioristik,[[0.19884443692969783]],Teori belajar behavioristik dikenal juga denga...,NaN,"[belajar, perilaku, teori, behavioristik]"


# Test SCRIPT STKI

In [56]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm

def search(teks):
  df = pd.read_excel(r'D:\TEST Codingan\Sistem-Temu-Kembali-Informasi\Riset Jupiter Notebook\Data\df_final.xlsx').head(1000)
  df = df.drop(columns='Unnamed: 0')

  teks = stem(teks)

  df_temu = []
  similaritas = []

  # Inisialisasi TF-IDF Vectorizer
  tfidf_vectorizer = TfidfVectorizer()

  for i in tqdm(range(len(df['text_stemmed']))):
    temp = df['text_stemmed'].iloc[i]

    # Transformasi teks menjadi vektor TF-IDF
    tfidf_matrix = tfidf_vectorizer.fit_transform([teks, temp])

    # Hitung similaritas kosinus antara kedua teks
    cosine_similarities = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

    if cosine_similarities[0][0] > 0.1:
      df_temu.append(df.iloc[i])
      similaritas.append(cosine_similarities)

  df_temu = pd.DataFrame(df_temu)
  df_temu['similaritas'] = similaritas

  return latent_semantic_analysis(df_temu)

def remove_stopwords(text):
  factory = StopWordRemoverFactory()
  stopwords = factory.get_stop_words()

  result = []
  for x in  text.split():
      if x not in stopwords:
          result.append(x)

  hasil_stopword = " ".join(result)

  return hasil_stopword

def stem(text):
  factory2 = StemmerFactory()
  stemmer = factory2.create_stemmer()
  return stemmer.stem(text)

def latent_semantic_analysis(dataset):
  tokenizer = RegexpTokenizer(r'\w+')
  topik = []

  for index in range(len(dataset)):
    # Ambil dataset
    teks = dataset['fulltext_stopword'].iloc[index]
    data = tokenizer.tokenize(teks)

    # Proses vektorisasi teks dengan TF-IDF
    tfidf_vectorizer = TfidfVectorizer(max_features=5000)
    tfidf_matrix = tfidf_vectorizer.fit_transform(data)

    # Terapkan LSA dengan TruncatedSVD
    num_topics = 4  # Jumlah topik yang ingin diidentifikasi
    lsa_model = TruncatedSVD(n_components=num_topics, random_state=42)
    lsa_topic_matrix = lsa_model.fit_transform(tfidf_matrix)

    # Ambil kata-kata kunci untuk setiap topik
    terms = tfidf_vectorizer.get_feature_names_out()
    top_keywords = 1  # Jumlah kata kunci untuk setiap topik (HANYA BISA DIAMBIL 1)

    kata_kunci = []

    for i, topic in enumerate(lsa_model.components_):
        top_keywords_idx = topic.argsort()[-top_keywords:][::-1]
        keywords = [terms[key_idx] for key_idx in top_keywords_idx]
        kata_kunci.append(keywords[0])

    topik.append(kata_kunci)
  dataset['topik'] = topik
  dataset = dataset.drop(columns=[
     'text_stemmed',
     'fulltext_stopword', 
     'fulltext', 
     'language_accuracy', 
     'language', 
     'text_raw', 
     'abstract', 
     'date',
     'keywords',
     'similaritas',
     'new_abstract',
     'publisher',
     'publication',
     'divisions'
    ])

  dataset = dataset.T

  hasil_pencarian = []
  for i in dataset.columns.values.tolist():
    hasil_pencarian.append(dataset[i].to_list())

  return hasil_pencarian
  # return dataset[1].tolist()
  # return dataset['title'].tolist()

In [57]:
df = search('Media Pembelajaran')
df

100%|██████████| 1000/1000 [00:04<00:00, 243.09it/s]


,1,37,44,49,135,154,208,233,267,282,...,893,902,925,928,929,930,937,959,998,999
divisions,['FKIP24'],['FMIPA2'],NaN,['FKIP18'],['FMIPA9'],['FMIPA2'],['FKIP12'],['FP25'],['FKIP22'],['FKIP24'],...,NaN,['FKIP19'],['FKIP24'],['FKIP15'],['FKIP15'],['FKIP2'],['FKIP16'],NaN,['FKIP22'],['FKIP22']
title,Media Pembelajaran Digital sebagai Sumber Bela...,Bedah Buku Ajar Biologi Kelas XII SMA pada Mat...,ANALISIS MATA KULIAH PRAKTIK SAAT PEMBELAJARAN...,Hak Cipta Buku Model Desain Pembelajaran,Perbandingan Pembelajaran Kesamaan Teks Menggu...,Peningkatan Kualitas Pemahaman Konten dan Tekn...,Peran Pembelajaran Pendidikan Pancasila dan Il...,PENGARUH VOLUME MEDIA TANAM DAN JUMLAH TANAMAN...,Pengaruh Kemandirian Belajar dan Literasi Digi...,Modul Pembelajaran Sejarah Tokoh Perjuangan La...,...,Pengaruh Bahan Ajar E-Modul Berpendekatan Cont...,Penggunaan Media Pop-Up Book terhadap Keteramp...,PELATIHAN PEMBELAJARAN DAN ASESMEN BERBASIS HO...,Kajian Faktor Intrinsik dan Kemampuan Literasi...,Pelatihan Penguatan Literasi dan Numerasi dala...,KAJIAN FAKTOR INTRINSIK DAN KEMAMPUAN LITERASI...,Teori Belajar Behavioristik,KEJADIAN TULI SENSORINEURAL DENGAN OTITIS MEDIA,PENGARUH KECERDASAN EMOSIONAL DAN KEMAMPUAN AW...,Pelatihan Pembelajaran Matematika Interaktif d...
subjects,['LA'],['QK'],['L1'],['G1'],['QA76'],"['QA', 'QA75', 'QC', 'QD', 'QK']","['L1', 'LB1603', 'LB2300', 'LB2361']",['S1'],['L1'],['LA'],...,['LB1603'],['L1'],['LA'],['L1'],['L1'],['L1'],['L1'],['R1'],['L1'],['L1']
publication,EDUKATIF : JURNAL ILMU PENDIDIKAN,Jurnal Pengabdi,"Geter : Jurnal Seni Drama, Tari dan Musik",NaN,NaN,Jurnal SOLMA,Educare: Jurnal Penelitian Pendidikan dan Pemb...,Jurnal Agrotropika,Jurnal Pendidikan Matematika Universitas Lampung,Modul Pembelajaran Sejarah Tokoh Perjuangan La...,...,Jurnal Pendidikan,TERAMPIL: Pendidikan dan Pembelajaran Dasar,JURNAL PENGABDIAN SOSIAL INDONESIA (Journal of...,LENSA (Lentera Sains): Jurnal Pendidikan IPA,NUWO-Abdimas,NaN,NaN,Jurnal Penelitian Perawat Profesional,AKSIOMA: Jurnal Program Studi Pendidikan Matem...,Ruang Pengabdian
publisher,Universitas Pahlawan Tuanku Tambusai,Lembaga Penelitian dan Pengabdian kepada Masya...,Jurusan Sendratasik FBS Unesa,NaN,NaN,Universitas Muhammadiyah Prof. DR. HAMKA,Actual Insight : Lembaga Penelitian Pengembang...,Jurnal Agrotropika,Fakultas Keguruan dan Ilmu Pendidikan Unversi...,Jurnal Education and Development,...,Universitas Terbuka,Program Studi Pendidikan Guru Madrasah Ibtidai...,Jurusan Pendidikan IPS,NaN,FKIP Universitas Lampung,Lensa (Lentera Sains) : Jurnal Pendidikan IPA,FKIP Universitas Lampung,NaN,"Mathematics Education, University of Muhammadi...",JPMIPA FKIP Universitas Lampung
creators,"[{'id': 'suparman.arif@fkip.unila.ac.id', 'nam...","[{'id': None, 'name': {'honourific': None, 'li...","[{'name': {'family': 'Daryanti', 'given': 'Fit...","[{'name': {'given': 'Mrs', 'lineage': None, 'f...","[{'name': {'given': 'Dian', 'lineage': None, '...","[{'name': {'honourific': None, 'lineage': None...","[{'name': {'family': 'Prasetyo', 'lineage': No...","[{'name': {'family': 'Rini', 'given': 'Maria V...","[{'name': {'given': 'Santy', 'lineage': None, ...","[{'id': 'sumargono.1988@fkip.unila.ac.id', 'na...",...,"[{'name': {'honourific': None, 'family': 'Lest...","[{'name': {'lineage': None, 'given': 'Loliyana...","[{'name': {'honourific': None, 'given': 'Masku...","[{'id': 'dina.maulina@fkip.unila.ac.id', 'name...","[{'id': 'dina.maulina@fkip.unila.ac.id', 'name...","[{'name': {'family': 'Maulina', 'given': 'Dina...","[{'name': {'honourific': None, 'given': 'Albet...","[{'name': {'lineage': None, 'given': 'MI', 'fa...","[{'id': 'hastuti_noer@yahoo.com', 'name': {'ho...","[{'id': 'hastuti_noer@yahoo.com', 'name': {'ho..."
uri,http://repository.lppm.unila.ac.id/id/eprint/4...,http://repository.lppm.unila.ac.id/id/eprint/5...,http://repository.lppm.unila.ac.id/id/eprint/5...,http://repository.lppm.unila.ac.id/id/eprint/5...,http://repository.lppm.unila.ac.id/id/eprint/5...,http://reposi

In [59]:
df.T.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 1 to 999
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   divisions     50 non-null     object
 1   title         59 non-null     object
 2   subjects      59 non-null     object
 3   publication   37 non-null     object
 4   publisher     45 non-null     object
 5   creators      59 non-null     object
 6   uri           59 non-null     object
 7   new_abstract  48 non-null     object
 8   topik         59 non-null     object
dtypes: object(9)
memory usage: 6.7+ KB
